In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import os,sys
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

# Custom packages
from wholebrain_tools import aba, genes, dataIO
import wholebrain_tools.stats as st 
import wholebrain_tools.graphics as gt

paths = dataIO.pathParser()
nodes_path = paths.structures
# Instantiate an Atlas object from the AbaTool.py file 
A = aba.Atlas(nodes=nodes_path)

DFM = aba.AnatomyDataFrameManager(A)

# Load data

In [ ]:
# ------------------------------------------------------------------------------
searchpath = paths.alldata
# ------------------------------------------------------------------------------

# Load dots data from all aniamls
df = dataIO.allMiceDots(searchpath)

# Select only PV cells that are on a non-zero region
df = df[df['pv']==1]
df = df[df['regionID']!=0]

df = DFM.add_resolution_column(df,'regionID')

df

# Visual areas

In [ ]:
pvDistrib = DFM.dots_to_sensorySystems(
        df,
        sensorySystem='visual',
        n_bins=4)
# Distribution of PV cells in intensity classes for each hierarchy
pvProportion = pvDistrib.divide(pvDistrib.sum(axis=1), axis=0)

# Actually plot data
gt.pvClasses_in_sensorySystems(
    pvProportion,
    legendTitle='Visual Areas',
    cmap='Purples'
    )

# Statistical analysis
_, _ = st.sensoryCortexIntClass(pvProportion)

# Optionally save the figure
# plt.savefig("VIS_pvIntensityClass.svg", bbox_inches="tight")

# Auditory areas

In [ ]:
pvDistrib = DFM.dots_to_sensorySystems(
        df,
        sensorySystem='auditory',
        n_bins=4)
# Distribution of PV cells in intensity classes for each hierarchy
pvProportion = pvDistrib.divide(pvDistrib.sum(axis=1), axis=0)

# Actually plot data
gt.pvClasses_in_sensorySystems(
    pvProportion,
    legendTitle='Auditory Areas',
    cmap='Purples'
    )

# Statistical analysis
_, _ = st.sensoryCortexIntClass(pvProportion)

# Optionally save the figure
# plt.savefig("AUD_pvIntensityClass.svg", bbox_inches="tight")

# Somatosensory areas

In [ ]:
pvDistrib = DFM.dots_to_sensorySystems(
        df,
        sensorySystem='somatosensory',
        n_bins=4)
# Distribution of PV cells in intensity classes for each hierarchy
pvProportion = pvDistrib.divide(pvDistrib.sum(axis=1), axis=0)

# Actually plot data
gt.pvClasses_in_sensorySystems(
    pvProportion,
    legendTitle='Somatosensory Areas',
    cmap='Purples'
    )

# Statistical analysis
_, _ = st.sensoryCortexIntClass(pvProportion)

# Optionally save the figure
# plt.savefig("SS_pvIntensityClass.svg", bbox_inches="tight")

# High - Low WFA areas

In [ ]:
# Get the annotation of cortical areas in subnetworks
subnet = dataIO.corticalAreasGroups(toDataFrame=True)
subnet.index = A.acronyms_to_ids(subnet.index)

# Add the high- and low- area groups  
group = ['hi' if x in ['motorSS','audioVisual','mAssociation'] else 'low' if x in ['lateral','mPrefrontal'] else 'other' for x in subnet['function']]
subnet['group'] = group

# Remove non-annotated ("other") cortical areas
subnet = subnet.loc[subnet['group'] != "other",:]

# Define the IDs of areas to keep as part of the selected sensory system
allIds = df['regionID'].unique()

targetIds = []
for putativeID in allIds:
    isDescendant = any([A.structure_descends_from(putativeID, x) for x in subnet.index])
    if isDescendant:
        targetIds.append(putativeID)

# Filter all areas in rawDf and keep only the ones is the selected sensory system
boolIdx = [x in targetIds for x in df['regionID']]
sensory_df = df.iloc[boolIdx,:]
# # Reset the index
sensory_df = sensory_df.reset_index()
sensory_df

In [ ]:
# Add group to the dataframe of all cells
highIDs = subnet[subnet['group']=='hi'].index
lowIDs = subnet[subnet['group']=='low'].index
group = []
for a in sensory_df['regionID']:
    isHigh = A.structure_descends_from_any(a, highIDs)
    isLow = A.structure_descends_from_any(a, lowIDs)
    if isHigh:
        group.append('high')
    elif isLow:
        group.append('low')
    else:
        raise
sensory_df['group'] = group

# Add info about the intensity class of PV cells 
n_bins = 4
edges = np.linspace(0,1,n_bins+1)
# Split PV cells in intensity classes
intensityClass, bins = pd.cut(
    sensory_df['fluoMeanPv'],
    edges,
    labels=[str(x+1) for x in range(n_bins)],
    retbins=True
)
sensory_df['intClass'] = intensityClass

# Count the number of PV cells for each mouse, hierarchy and class
count_df = sensory_df.groupby(['mouse', 'group', 'intClass']).count()['pv']
count_df = count_df.unstack('intClass')
count_df

In [ ]:
# Distribution of PV cells in intensity classes for each hierarchy
pvProportion = count_df.divide(count_df.sum(axis=1), axis=0)

# Actually plot data
gt.pvClasses_in_sensorySystems(
    pvProportion,
    hueVar='group',
    legendTitle='Cortical subnetworks',
    cmap='Purples'
    )

# Statistical analysis
_, _ = st.hiLowWfa_IntClass(pvProportion)

# plt.savefig("hilowWFA_pvIntensityClass.svg", bbox_inches="tight")